In [43]:
import requests
import json

In [44]:
#AC_URL = "http://localhost:6380/query"
AC_URL = "https://answercoalesce.renci.org/query"

In [45]:
def generate_trapi_query(input_node_type,output_node_type,input_curies,predicate,input_is_subject=True):
    envelope = {"message":{"query_graph": {"nodes":{"input":{},"output":{}}, "edges":{"edge_0":{}}}}}
    input_node = envelope["message"]["query_graph"]["nodes"]["input"]
    input_node["categories"] = [input_node_type]
    input_node["ids"] = ["uuid:1"]
    input_node["member_ids"] = input_curies
    input_node["set_interpretation"]="MANY"
    output_node = envelope["message"]["query_graph"]["nodes"]["output"]
    output_node["categories"] = [output_node_type]
    query_edge = envelope["message"]["query_graph"]["edges"]["edge_0"]
    if input_is_subject:
        query_edge["subject"] = "input"
        query_edge["object"] = "output"
    else:
        query_edge["subject"] = "output"
        query_edge["object"] = "input"
    query_edge["predicates"]  = [ predicate ]
    #query_edge["knowledge_type"] = "inferred"
    return envelope

In [46]:
def phenotypes_to_disease(phenotypes, print_query=False):
    #Phenotype to Disease
    input_type = "biolink:PhenotypicFeature"
    output_type = "biolink:Disease"
    predicate = "biolink:has_phenotype"
    input_is_subject = "False"
    query = generate_trapi_query( input_type, output_type, phenotypes, predicate, input_is_subject)
    if print_query:
        print(json.dumps(query,indent=2))
    response = requests.post( AC_URL, json = query )
    print(response.status_code)
    return response.json()

In [47]:
def genes_to_chemical(genes, print_query=False):
    #Phenotype to Disease
    input_type = "biolink:Gene"
    output_type = "biolink:ChemicalEntity"
    predicate = "biolink:related_to"
    input_is_subject = "False"
    query = generate_trapi_query( input_type, output_type, genes, predicate, input_is_subject)
    if print_query:
        print(json.dumps(query,indent=2))
    response = requests.post( AC_URL, json = query )
    print(response.status_code)
    return response.json()

In [48]:
def phenotypes_to_gene(phenotypes):
    #Phenotype to Gene
    input_type = "biolink:PhenotypicFeature"
    output_type = "biolink:Gene"
    predicate = "biolink:genetically_associated_with"
    input_is_subject = "True"
    query = generate_trapi_query( input_type, output_type, phenotypes, predicate, input_is_subject)
    response = requests.post( AC_URL, json = query )
    print(response.status_code)
    return response.json()

In [49]:
def print_results(resp):
    for result in resp["message"]["results"]:
        nb = result["node_bindings"]["output"][0]["id"]
        name = resp["message"]["knowledge_graph"]["nodes"][nb]["name"]
        edge_id=result["analyses"][0]["edge_bindings"]["edge_0"][0]["id"]
        edge_score=result["analyses"][0]["score"]
        edge = resp["message"]["knowledge_graph"]["edges"][edge_id]
        p=""
        edge_atts = edge["attributes"]
        for att in edge_atts:
            if att["attribute_type_id"] == "biolink:p_value":
                p = att["value"]
        print(nb, name, p, edge_score) 

In [50]:
genes = ["NCBIGene:5297", "NCBIGene:5298","NCBIGene:5290"]
r = genes_to_chemical(genes, print_query=False)

200


In [51]:
print_results(r)

PUBCHEM.COMPOUND:121452612 5-[2-[[(2R)-1-cyclohexylpropan-2-yl]amino]-4-methyl-1,3-thiazol-5-yl]-2-methoxy-N-piperidin-4-ylbenzenesulfonamide 1.3427467469050326e-14 0.988295
PUBCHEM.COMPOUND:5109813 6-Phenylbenzo[d]thiazol-2-amine 1.3427467469050326e-14 0.988295
PUBCHEM.COMPOUND:122184478 5-(2-amino-1,3-benzothiazol-6-yl)-N-methylpyridine-2-carboxamide 1.3427467469050326e-14 0.988295
PUBCHEM.COMPOUND:121452677 5-[2-[[(1R)-1-cyclohexylethyl]amino]-4-methyl-1,3-thiazol-5-yl]-2-methoxy-N-piperidin-4-ylbenzenesulfonamide 1.3427467469050326e-14 0.988295
PUBCHEM.COMPOUND:122184472 [5-(2-Amino-1,3-benzothiazol-6-yl)pyridin-2-yl]methanol 1.3427467469050326e-14 0.988295
PUBCHEM.COMPOUND:89493897 5-[2-(cycloheptylamino)-4-methyl-1,3-thiazol-5-yl]-N-(2,6-dimethylphenyl)-2-methoxybenzenesulfonamide 1.3427467469050326e-14 0.988295
PUBCHEM.COMPOUND:121467267 5-[2-[[(1R)-1-cyclohexylethyl]amino]-4-methyl-1,3-thiazol-5-yl]-2-methoxy-N-piperidin-3-ylbenzenesulfonamide 1.3427467469050326e-14 0.988295
PU

In [52]:
#Leigh Disease?
leigh_phenotypes = ["HP:0000739", # | Anxiety, 
                    "HP:0001288", # | Gait disturbance, 
                    "HP:0001252", # | Hypotonia, 
                    "HP:0001250", # | Seizure, 
                    "HP:0000750", # | Delayed speech and language development, 
                    "HP:0002378", # | Hand tremor, 
                    "HP:0002019", # | Constipation, 
                    "HP:0007146"] # | Bilateral basal ganglia lesions

In [53]:
input_type = "biolink:PhenotypicFeature"
output_type = "biolink:Disease"
predicate = "biolink:has_phenotype"
input_is_subject = "False"
query = generate_trapi_query( input_type, output_type, leigh_phenotypes, predicate, input_is_subject)
print(json.dumps(query,indent=2))

{
  "message": {
    "query_graph": {
      "nodes": {
        "input": {
          "categories": [
            "biolink:PhenotypicFeature"
          ],
          "ids": [
            "uuid:1"
          ],
          "member_ids": [
            "HP:0000739",
            "HP:0001288",
            "HP:0001252",
            "HP:0001250",
            "HP:0000750",
            "HP:0002378",
            "HP:0002019",
            "HP:0007146"
          ],
          "set_interpretation": "MANY"
        },
        "output": {
          "categories": [
            "biolink:Disease"
          ]
        }
      },
      "edges": {
        "edge_0": {
          "subject": "input",
          "object": "output",
          "predicates": [
            "biolink:has_phenotype"
          ]
        }
      }
    }
  }
}


In [54]:
response = phenotypes_to_disease(leigh_phenotypes, print_query=False)
print_results(response)

200
MONDO:0859085 neurodevelopmental disorder with gait disturbance, dysmorphic facies, and behavioral abnormalities, X-linked 3.676409906489578e-09 0.847779
MONDO:0014553 Tenorio syndrome 1.547709027607439e-08 0.803001
MONDO:0060761 neurodevelopmental disorder with epilepsy and hypoplasia of the corpus callosum 1.547709027607439e-08 0.803001
MONDO:0013761 childhood encephalopathy due to thiamine pyrophosphokinase deficiency 1.779218036784196e-08 0.798168
MONDO:0010496 X-linked intellectual disability-short stature-overweight syndrome 2.0327049157737718e-08 0.793469
MONDO:0012342 7q11.23 microduplication syndrome 3.664489074574276e-08 0.77171
MONDO:0010305 creatine transporter deficiency 6.554722689681547e-08 0.748706
MONDO:0008939 isolated cerebellar hypoplasia/agenesis 3.726313896403495e-07 0.671364
MONDO:0012423 MORM syndrome 3.726313896403495e-07 0.671364
MONDO:0014953 gnb5-related intellectual disability-cardiac arrhythmia syndrome 5.82152762230098e-07 0.649646
MONDO:0030036 leuko

In [55]:
response = phenotypes_to_gene(leigh_phenotypes)
print_results(response)

200
NCBIGene:93986 FOXP2 0.0006474412355706435 0.287901
NCBIGene:53335 BCL11A 0.0006474412355706435 0.287901
NCBIGene:85358 SHANK3 0.0008631618162098167 0.27527
NCBIGene:2077 ERF 0.0008631618162098167 0.27527
NCBIGene:9378 NRXN1 0.0008631618162098167 0.27527
NCBIGene:1806 DPYD 0.0010788358313316428 0.265714
NCBIGene:26047 CNTNAP2 0.0015100442052276759 0.251713
NCBIGene:26040 SETBP1 0.0015100442052276759 0.251713
NCBIGene:284058 KANSL1 0.001725578584098671 0.246296
NCBIGene:50484 RRM2B 0.0019410664376458965 0.241583
NCBIGene:113189 CHST14 0.0019410664376458965 0.241583
NCBIGene:2903 GRIN2A 0.002156507775912319 0.23742
NCBIGene:5728 PTEN 0.003017808176953609 0.224463
NCBIGene:3737 KCNA2 0.004093388288691872 0.21315
NCBIGene:3954 LETM1 0.00495301684843197 0.20629
NCBIGene:4204 MECP2 0.008170020749701742 0.189063
NCBIGene:18 ABAT 0.011376624022382093 0.178286


In [56]:
leigh_2 = [ "HP:0001252", #	""Hypotonia"""
            "HP:0002521", #	""Hypsarrhythmia"""
            "HP:0001263", #	""Global developmental delay"""
            "HP:0004305", #	""Involuntary movements"""
            "HP:0002579", #	""Gastrointestinal dysmotility"""
            "HP:0200147", # "Neuronal loss in basal ganglia". 
            "HP:0007183" #	""Focal T2 hyperintense basal ganglia lesion"""
          ]

In [57]:
response = phenotypes_to_disease(leigh_2)
print_results(response)

200
MONDO:0030919 intellectual disability, autosomal dominant 53 5.536048099918041e-11 0.932667
MONDO:0014942 developmental and epileptic encephalopathy, 45 2.4244693060343317e-10 0.909512
MONDO:0014842 intellectual disability, autosomal dominant 41 3.849422096046302e-10 0.900903
MONDO:0014477 developmental and epileptic encephalopathy, 26 1.4929091468683857e-09 0.871354
MONDO:0011966 periventricular heterotopia with microcephaly, autosomal recessive 1.4929091468683857e-09 0.871354
MONDO:0013164 beta-ureidopropionase deficiency 1.4929091468683857e-09 0.871354
MONDO:0030963 Li-Campeau syndrome 1.9379275651635485e-09 0.864869
MONDO:0033366 developmental and epileptic encephalopathy, 57 1.9379275651635485e-09 0.864869
MONDO:0010657 methylmalonic acidemia with homocystinuria, type cblX 1.9379275651635485e-09 0.864869
MONDO:0011964 DPAGT1-congenital disorder of glycosylation 2.4635554288901497e-09 0.858662
MONDO:0033362 developmental and epileptic encephalopathy, 53 3.076485468182307e-09 0.

In [58]:
response = phenotypes_to_gene(leigh_2)
print_results(response)

200
NCBIGene:51733 UPB1 0.0001888803437402939 0.345678
NCBIGene:3295 HSD17B4 0.0003777250116963364 0.312472
NCBIGene:84152 PPP1R1B 0.0009440450282371355 0.271407
NCBIGene:284058 KANSL1 0.0015100442052276759 0.251713
NCBIGene:50484 RRM2B 0.0016986393312994233 0.246931
NCBIGene:113189 CHST14 0.0016986393312994233 0.246931
NCBIGene:1356 CP 0.0033943932870210093 0.220049
NCBIGene:3356 HTR2A 0.0033943932870210093 0.220049
NCBIGene:7249 TSC2 0.0037708361514733027 0.216154
NCBIGene:7248 TSC1 0.0037708361514733027 0.216154
NCBIGene:3954 LETM1 0.004335233855151651 0.211067
NCBIGene:4204 MECP2 0.0071524297375753 0.193531
NCBIGene:4988 OPRM1 0.007339959127928182 0.192656
NCBIGene:1813 DRD2 0.008277074904243129 0.18863
NCBIGene:1392 CRH 0.008464391871230346 0.187887
NCBIGene:5173 PDYN 0.008651673457724452 0.187163
NCBIGene:18 ABAT 0.009961654470562097 0.18255
NCBIGene:5443 POMC 0.013694926855229318 0.172462


In [59]:
phenotypes_2 = [
    "HP:0000718", # aggression         | 2 yrs - present
    "HP:0000739", #anxiety                | 2 yrs - present
    "HP:0002616", # Aortic root aneurysm HP:0002616 | 10 yrs - present
    "HP:0002019", #constipation (occasional) HP:0002019 |        birth - present
    #occasional bright lime-green stool | 5 yrs - present
    "HP:0000678", #crowded teeth        HP:0000678         
    "HP:0000886", # Deformed rib cage HP:0000886 | 6 months - present
    "HP:0002046", #heat intolerance        HP:0002046 
    "HP:0002705", #high arched palate        HP:0002705 
    "HP:0001290", #hypotonia        HP:0001290 |        birth - present
    "HP:0000737", # irritable        HP:0000737 |        2 yrs - present
    "HP:0001382", # Joint hypermobility HP:0001382 | birth - present
    "HP:0000720", # mood swings        HP:0000720 |        2 yrs - present
    "HP:0012801", # narrow jaw        HP:0012801
    "HP:0010865", # Oppositional defiant disorder HP:0010865 | 2 yrs - present
    #Destructive tendencies | 3 yrs - present
    "HP:0002172", # Postural instability HP:0002172 | birth - present
    "HP:0001250", # seizures        HP:0001250 |        7 yrs - present
    "HP:0001649", # Tachycardia HP:0001649 | 1 yr - present
    "HP:0001337", # tremor        HP:0001337 | in hands 11 yrs - present
    #Vascular skin abnormality (superficial blood vessels that rupture easily) HP:0011276 |  birth - present
    "HP:0001123" # Visual field defect HP:0001123 | birth - present
    # Seasonal allergies | birth - present
    #Food allergy | 4 yrs - present
]

In [60]:
response = phenotypes_to_disease(phenotypes_2)
print_results(response)

200
MONDO:0010501 syndromic X-linked intellectual disability 34 2.342808145498639e-13 0.97844
MONDO:0010441 CK syndrome 3.4695918945210576e-12 0.961943
MONDO:0010306 X-linked intellectual disability, Cabezas type 2.012562565025918e-09 0.863907
MONDO:0010671 microphthalmia, syndromic 1 2.303903948394455e-09 0.860418
MONDO:0010655 X-linked intellectual disability with marfanoid habitus 4.259526562078124e-09 0.843608
MONDO:0010500 intellectual disability, X-linked, syndromic 33 1.763189694158661e-08 0.798485
MONDO:0032605 intellectual disability, autosomal recessive 66 2.237839556363733e-08 0.790027
MONDO:0014606 intellectual disability-microcephaly-strabismus-behavioral abnormalities syndrome 2.576201852588367e-08 0.78491
MONDO:0859143 Radio-Tartaglia syndrome 2.7675996639343947e-08 0.782271
MONDO:0019349 Sotos syndrome 2.9694267590458545e-08 0.779656
MONDO:0958179 glycine encephalopathy 1 4.9108665950578294e-08 0.760317
MONDO:0013933 peroxisome biogenesis disorder 5B 7.538205405614794e-

In [61]:
response = phenotypes_to_gene(phenotypes_2)
print_results(response)

200
NCBIGene:7200 TRH 0.00043350422688401063 0.306083
NCBIGene:6262 RYR2 0.0010249216925187008 0.267892
NCBIGene:6343 SCT 0.0010249216925187008 0.267892
NCBIGene:10451 VAV3 0.002048792920561606 0.239441
NCBIGene:3360 HTR4 0.0025603349468989512 0.230738
NCBIGene:51083 GAL 0.005624085345056082 0.201809
NCBIGene:3972 LHB 0.005624085345056082 0.201809
NCBIGene:2778 GNAS 0.011216540354143694 0.178737
NCBIGene:2796 GNRH1 0.011216540354143694 0.178737
NCBIGene:1132 CHRM4 0.011216540354143694 0.178737
NCBIGene:7432 VIP 0.012736289207362732 0.174722
NCBIGene:196 AHR 0.01324235290623453 0.173506
NCBIGene:2641 GCG 0.01324235290623453 0.173506
NCBIGene:3827 KNG1 0.01475898810876207 0.170155
NCBIGene:154 ADRB2 0.01475898810876207 0.170155
NCBIGene:3952 LEP 0.017785268943805686 0.164512
NCBIGene:43 ACHE 0.020300067645087804 0.160603
NCBIGene:6334 SCN8A 0.020300067645087804 0.160603
NCBIGene:1813 DRD2 0.022307269930771232 0.157862
NCBIGene:1565 CYP2D6 0.022307269930771232 0.157862
NCBIGene:590 BCHE 0

In [62]:
tbcd_phenotypes=[
    "HP:0002098", # | Respiratory distress, 
    "HP:0001252", # | Hypotonia, 
    "HP:0001250" # | Seizure
]

In [63]:
response = phenotypes_to_disease(tbcd_phenotypes)
print_results(response)

200
MONDO:0014271 STT3B-congenital disorder of glycosylation 2.096353238263982e-07 0.698311
MONDO:0014856 combined oxidative phosphorylation defect type 30 5.534189247952736e-07 0.652143
MONDO:0010490 SSR4-congenital disorder of glycosylation 5.534189247952736e-07 0.652143
MONDO:0008056 myotonic dystrophy type 1 5.534189247952736e-07 0.652143
MONDO:0012792 mitochondrial DNA depletion syndrome 8a 7.367208238964271e-07 0.63792
MONDO:0014754 primary coenzyme Q10 deficiency 8 8.381793480638825e-07 0.631424
MONDO:0859279 spinal muscular atrophy, distal, autosomal recessive, 6 9.461748430404016e-07 0.625279
MONDO:0013307 myopathy, lactic acidosis, and sideroblastic anemia 2 1.060706249785143e-06 0.619447
MONDO:0060704 neurodevelopmental disorder with spastic quadriplegia and brain abnormalities with or without seizures 1.060706249785143e-06 0.619447
MONDO:0009612 methylmalonic aciduria due to methylmalonyl-CoA mutase deficiency 1.060706249785143e-06 0.619447
MONDO:0010271 X-linked myotubular

In [64]:
response = phenotypes_to_gene(tbcd_phenotypes)
print_results(response)

200
NCBIGene:284058 KANSL1 0.0006474412355706435 0.287901
NCBIGene:50484 RRM2B 0.0007283419108450321 0.282688
NCBIGene:113189 CHST14 0.0007283419108450321 0.282688
NCBIGene:3954 LETM1 0.001860263944114525 0.243279
NCBIGene:4204 MECP2 0.003071614760772548 0.223795
NCBIGene:18 ABAT 0.004281495471845135 0.211518


In [65]:
wdyf_phenotypes = [
    #Global developmental delay 
    "HP:0001263",
    #Seizure 
    "HP:0001250",
    #Neurodevelopmental delay 
    "HP:0012758",
    #Delayed social development 
    "HP:0012434"
]

In [66]:
response = phenotypes_to_disease(wdyf_phenotypes)
print_results(response)

200
MONDO:0012830 chromosome 10q23 deletion syndrome 7.043541590830241e-07 0.640171
MONDO:0014992 lissencephaly 8 1.489667817869212e-06 0.601916
MONDO:0032866 cortical dysplasia, complex, with other brain malformations 10 1.8850895473043174e-06 0.589605
MONDO:0014908 microcephaly 17, primary, autosomal recessive 2.32693618090683e-06 0.578497
MONDO:0060761 neurodevelopmental disorder with epilepsy and hypoplasia of the corpus callosum 2.5652625674495586e-06 0.573326
MONDO:0060758 spinocerebellar ataxia 42, early-onset, severe, with neurodevelopmental deficits 2.8151876576608396e-06 0.56838
MONDO:0029143 intellectual developmental disorder with hypertelorism and distinctive facies 3.076708944928911e-06 0.563642
MONDO:0014351 pontocerebellar hypoplasia type 9 3.3498239230479854e-06 0.559095
MONDO:0033492 Coffin-Siris syndrome 6 4.5581706341248245e-06 0.542548
MONDO:0014601 autosomal recessive spinocerebellar ataxia 20 5.951816007870529e-06 0.528139
MONDO:0013076 attention deficit-hyperact

In [67]:
# response = phenotypes_to_gene(wdyf_phenotypes)
# print_results(response)

# No enrichment in the Database

In [68]:
twogene_phenotypes = [
    #Recurrent fever 
    "HP:0001954",
    #hip dysplasia        
    "HP:0001385",
    #joint hypermobility        
    "HP:0001382",
    #optic disc pallor        
    "HP:0000543",        
    #recurrent infections        
    "HP:0002719",        
    #thin corpus callosum        
    "HP:0002079",
    #Global developmental delay 
    "HP:0001263",
    #Abnormality of vision 
    "HP:0000504",
    #Nystagmus 
    "HP:0000639",
    #Drooling 
    "HP:0002307",
    #Dysphagia 
    "HP:0002015",
    #Hypotonia 
    "HP:0001252"
    #Eczema 
    "HP:0000964",
    #Inability to walk 
    "HP:0002540",
    #Septo-optic dysplasia 
    "HP:0100842"
    #Postural instability 
    "HP:0002172"
]

In [69]:
response = phenotypes_to_disease(twogene_phenotypes)
print_results(response)

200
MONDO:0030990 Kohlschutter-Tonz syndrome-like 3.267366897642553e-13 0.976863
MONDO:0032781 congenital hypotonia, epilepsy, developmental delay, and digital anomalies 1.5438766550103143e-12 0.967882
MONDO:0010533 Arts syndrome 1.7537587683174986e-11 0.946749
MONDO:0010283 syndromic X-linked intellectual disability Lubs type 2.177644306855737e-11 0.944329
MONDO:0014746 SLC39A8-CDG 2.4148326614321293e-11 0.943136
MONDO:0033371 developmental and epileptic encephalopathy, 62 2.4148326614321293e-11 0.943136
MONDO:0014905 encephalopathy due to defective mitochondrial and peroxisomal fission 2 4.2777578280153676e-11 0.936099
MONDO:0032710 developmental and epileptic encephalopathy, 72 5.536048099918041e-11 0.932667
MONDO:0014979 myoclonus, intractable, neonatal 7.053170414013553e-11 0.929288
MONDO:0044701 childhood-onset motor and cognitive regression syndrome with extrapyramidal movement disorder 8.862321093680521e-11 0.92596
MONDO:0011897 leukoencephalopathy-ataxia-hypodontia-hypomyelina

In [70]:
response = phenotypes_to_gene(twogene_phenotypes)
print_results(response)

200
NCBIGene:84466 MEGF10 0.0011327470604279887 0.263653
NCBIGene:79042 TSEN34 0.0037708361514733027 0.216154
NCBIGene:283989 TSEN54 0.004147136824040217 0.212675
NCBIGene:80746 TSEN2 0.0045232953584311885 0.209536
NCBIGene:6863 TAC1 0.012016699562439063 0.176551


In [71]:
autism_phenotypes=[
    #Autistic behavior 
    "HP:0000729",
    #Neurodevelopmental delay 
    "HP:0012758",
    #Intellectual disability 
    "HP:0001249",
    #Ventricular septal defect 
    "HP:0001629",
    #Abnormal facial shape 
    "HP:0001999",
    #high arched palate        
    "HP:0002705",
    #high nasal bridge        
    "HP:0000426",
    #shallow orbits        
    "HP:0000586",
    #Abnormality of the palmar creases 
    "HP:0010490"
]

In [72]:
response = phenotypes_to_disease(autism_phenotypes)
print_results(response)

200
MONDO:0033492 Coffin-Siris syndrome 6 5.214647608277274e-08 0.757934
MONDO:0013256 chromosome 15q24 deletion syndrome 1.7122897278399835e-06 0.594647
MONDO:0008736 peroxisome biogenesis disorder 2B 4.23870594654196e-06 0.546461
MONDO:0032850 neurooculocardiogenitourinary syndrome 6.61976266806481e-06 0.52238
MONDO:0859252 neurodevelopmental disorder with poor growth and skeletal anomalies 1.422372092350377e-05 0.480883
MONDO:0030030 Nizon-Isidor syndrome 2.3017716165030653e-05 0.454865
MONDO:0008999 Cohen syndrome 3.592446802548804e-05 0.431019
MONDO:0030679 Noonan syndrome 14 3.800047203182352e-05 0.42803
MONDO:0014831 progeroid and marfanoid aspect-lipodystrophy syndrome 3.800047203182352e-05 0.42803
MONDO:0010664 syndromic X-linked intellectual disability Snyder type 4.457613506478945e-05 0.419567
MONDO:0009318 Hallermann-Streiff syndrome 5.415381663330505e-05 0.409311
MONDO:0014700 Au-Kline syndrome 5.928934823615599e-05 0.404563
MONDO:0011150 acroosteolysis-keloid-like lesions

In [73]:
# response = phenotypes_to_gene(autism_phenotypes)
# print_results(response)

# No enrichment in the database